In [ ]:
import cv2
import time
import numpy as np
import random
"""
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
"""

protoFile = "hand/pose_deploy.prototxt"
weightsFile = "hand/pose_iter_102000.caffemodel"
nPoints = 22
POSE_PAIRS = [ [0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20] ]

threshold = 0.3

#Enter input filename
input_source = "fist_dorsal.webm"
cap = cv2.VideoCapture(input_source)
Randomframes=[]
point_dict={}
point_dict["x"]={}
point_dict["y"]={}
max_frame=0

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
k = 0
Storeframe=[]
hasFrame, frame = cap.read()

#Capture all frames of a video
while hasFrame:
    hasFrame, frame = cap.read()
    Storeframe.append(frame)
    max_frame = max_frame+1

#Selecting random frames
for x in range(100):
    Randomframes.append(random.randint(0,max_frame) )       

k=0 #Counter to just update dictionary for now.

#Start landmark extraction only on selected frames
for i in Randomframes:
    k=k+1
    t = time.time()
    #hasFrame, frame = cap.read()
    frame = Storeframe[i]
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (640,480), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()
    #print("forward = {}".format(time.time() - t))

    # Empty list to store the detected keypoints
    points = []
    for i in range(nPoints):
        if i==1 or i==21: #to skip the extra point in the trained model
            continue
            
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (640, 480))

        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        if prob > threshold :
            #Uncomment the below line to see output
            cv2.circle(frame, (int(point[0]), int(point[1])), 6, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
            points.append((int(point[0]), int(point[1])))
            point_dict["x"][k]=int(point[0])
            point_dict["y"][k]=int(point[1])
        else :
            points.append(None)

    #print("Time Taken for frame {} = {}".format(k,time.time() - t))
    
    #Uncomment the below line to see output
    cv2.imshow('Output-Skeleton', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

    print("total = {}".format(time.time() - t))
    
from pprint import pprint
pprint(point_dict)
# Save workbook to write
#wb.save(filepath)